In [1]:
#Import necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost.sklearn import XGBRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import RBF
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1
from tensorflow.keras.layers import Dense, Dropout
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics
import seaborn as sns
from keras import backend as K

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-12-16 17:45:45.929329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Function for regression metrics

def regression_results(y_true, y_pred):
    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)
    print('explained_variance: ', round(explained_variance,4))    
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))
    
def coeff_determination(y_true, y_pred):
    SS_res = K.sum(K.square(y_true-y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

In [3]:
#Import the data and create a DataFrame

data1 = pd.read_csv("training_dataset.csv") 
features = data1.iloc[:, [0, 1, 2, 3, 4, 5]]
output = data1.iloc[:, 6]

          PHI         T         P       NH3        H2       CH4       LBV
PHI  1.000000  0.092599  0.244596  0.076909  0.179916 -0.236934  0.222494
T    0.092599  1.000000  0.343669  0.090281  0.142880 -0.215604  0.261223
P    0.244596  0.343669  1.000000  0.213079  0.346385 -0.517244  0.087909
NH3  0.076909  0.090281  0.213079  1.000000 -0.418946 -0.556186 -0.500210
H2   0.179916  0.142880  0.346385 -0.418946  1.000000 -0.521598  0.720978
CH4 -0.236934 -0.215604 -0.517244 -0.556186 -0.521598  1.000000 -0.189864
LBV  0.222494  0.261223  0.087909 -0.500210  0.720978 -0.189864  1.000000


,PHI,T,P,NH3,H2,CH4,LBV
0,0.5,298.0,1.0,80.0,20.0,0.0,4.78
1,0.5,298.0,1.0,20.0,80.0,0.0,43.60
2,0.5,298.0,1.0,0.0,100.0,0.0,67.20
3,0.5,298.0,1.0,0.0,100.0,0.0,66.30
4,0.5,298.0,1.0,0.0,100.0,0.0,42.00


In [5]:
# Scale the feature vectors

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

/Users/cihatemreustun/opt/anaconda3/envs/ct-env/lib/python3.9/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/cihatemreustun/opt/anaconda3/envs/ct-env/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/cihatemreustun/opt/anaconda3/envs/ct-env/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/cihatemreustun/opt/anaconda3/envs/ct-env/lib/python3.9/site-packages/sklearn/utils/validation.py:767: Fu

In [7]:
X = scaled_features
y = output

plots = []
plots2 = []
tests = []

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)
tests.append(y_test)

model = keras.Sequential()
model.add(Dense(45, activation='relu', input_shape=[6]))
model.add(Dense(45, activation='relu'))
model.add(Dense(45, activation='relu'))
model.add(Dense(45, activation='relu'))
model.add(Dense(45, activation='relu'))
model.add(Dense(45, activation='relu'))
model.add(Dense(45, activation='relu'))
model.add(Dense(45, activation='relu'))
model.add(Dense(45, activation='relu'))
model.add(Dense(45, activation='relu'))
model.add(Dense(1))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4,decay=1e-4), 
              loss='mae', 
              metrics=[coeff_determination])
    
history = model.fit(X_train, y_train, batch_size=128, epochs=2500, validation_data=(X_val, y_val))

plots.append(model.predict(X_test))
plots2.append(model.predict(X_train))
y_pred_ann = model.predict(X_test)
loss_train = history.history['loss']
loss_val = history.history['val_loss']
accuracy = history.history['coeff_determination']
epochs = range(1,2501)

2023-12-16 17:45:49.317556: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/2500
41/41 [==============================] - 1s 5ms/step - loss: 70.0972 - coeff_determination: -0.5779 - val_loss: 72.8140 - val_coeff_determination: -0.6734
Epoch 2/2500
41/41 [==============================] - 0s 2ms/step - loss: 66.0967 - coeff_determination: -0.5089 - val_loss: 59.6071 - val_coeff_determination: -0.3712
Epoch 3/2500
41/41 [==============================] - 0s 2ms/step - loss: 51.8298 - coeff_determination: -0.0764 - val_loss: 49.7640 - val_coeff_determination: -0.0079
Epoch 4/2500
41/41 [==============================] - 0s 2ms/step - loss: 43.4499 - coeff_determination: 0.1718 - val_loss: 38.7383 - val_coeff_determination: 0.3095
Epoch 5/2500
41/41 [==============================] - 0s 2ms/step - loss: 31.5847 - coeff_determination: 0.5316 - val_loss: 26.0249 - val_coeff_determination: 0.6681
Epoch 6/2500
41/41 [==============================] - 0s 2ms/step - loss: 22.7639 - coeff_determination: 0.7551 - val_loss: 18.7630 - val_coeff_determination: 0.785

In [8]:
# Assessing the model accuracy

regression_results(y_test, y_pred_ann)

explained_variance:  0.9923
r2:  0.9923
MAE:  2.2228
MSE:  60.4559
RMSE:  7.7753


/Users/cihatemreustun/opt/anaconda3/envs/ct-env/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/cihatemreustun/opt/anaconda3/envs/ct-env/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/cihatemreustun/opt/anaconda3/envs/ct-env/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/cihatemreustun/opt/anaconda3/envs/ct-env/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be 

In [11]:
# Making new predictions

data2 = pd.read_csv("Pred_matrix_all.csv") 
scaled_data2 = scaler.fit_transform(data2)
scaled_data2 = pd.DataFrame(scaled_data2)

ternary = []
ternary.append(model.predict(scaled_data2))
print(ternary)

88/88 [==============================] - 0s 532us/step
[array([[  8.622741],
       [  9.258625],
       [  9.749113],
       ...,
       [ 49.743214],
       [684.71716 ],
       [ 28.005508]], dtype=float32)]


/Users/cihatemreustun/opt/anaconda3/envs/ct-env/lib/python3.9/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/cihatemreustun/opt/anaconda3/envs/ct-env/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/cihatemreustun/opt/anaconda3/envs/ct-env/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/cihatemreustun/opt/anaconda3/envs/ct-env/lib/python3.9/site-packages/sklearn/utils/validation.py:767: Fu

In [12]:
# Writing the predictions into a CSV file

print(ternary)
import csv

with open('Predictions_all.csv', 'w') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
     
    write.writerows(ternary)

[array([[  8.622741],
       [  9.258625],
       [  9.749113],
       ...,
       [ 49.743214],
       [684.71716 ],
       [ 28.005508]], dtype=float32)]


In [ ]:
# XGBoost predictions

pipe1 = Pipeline([('scaled', StandardScaler()), ('LR', XGBRegressor())])
pipe1.fit(X_train, y_train)
y_pred_xgboost = pipe1.predict(X_test)
print(pipe1.score(X_test, y_test))

In [ ]:
# Assessing the model accuracy

regression_results(y_test, y_pred_xgboost)

In [ ]:
tests_xgboost = []
plots_xgboost = []
plots2_xgboost = []
tests_xgboost.append(y_test)
plots_xgboost.append(pipe1.predict(X_test))
plots2_xgboost.append(pipe1.predict(X_train))

In [ ]:
# Train GPR

# Different kernels to test

rbfkernel = 1.0 * RBF(length_scale=0.1, length_scale_bounds=(1e-1, 10.0))
expkernel = 1.0 * ExpSineSquared(
    length_scale=1.0,
    periodicity=3.0,
    length_scale_bounds=(0.1, 10.0),
    periodicity_bounds=(1.0, 10.0),
)

quadkernel = 1.0 * RationalQuadratic(length_scale=0.1, alpha=0.1, alpha_bounds=(1e-5, 1e15))
maternkernel = 1.0 * Matern(length_scale=0.1, length_scale_bounds=(1e-1, 10.0), nu=1.5)

gpr = GaussianProcessRegressor(kernel = expkernel,random_state=0).fit(X_train, y_train)
print(gpr.get_params())
#print(f"Kernel parameters before fit:\n{gpr.kernel_})")
print(
    f"Kernel parameters after fit: \n{gpr.kernel_} \n"
    f"Log-likelihood: {gpr.log_marginal_likelihood(gpr.kernel_.theta):.3f}")

In [ ]:
tests_gpr = []
plots_gpr = []
plots2_gpr = []
tests_gpr.append(y_test)
plots_gpr.append(pipe2.predict(X_test))
plots2_gpr.append(pipe2.predict(X_train))

In [ ]:
print(gpr.get_params())

In [ ]:
print(f"Kernel parameters before fit:\n{kernel})")
print(
    f"Kernel parameters after fit: \n{gpr.kernel_} \n"
    f"Log-likelihood: {gpr.log_marginal_likelihood(gpr.kernel_.theta):.3f}")

In [ ]:
regression_results(y_test, y_pred)